In [5]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import os
import csv
import textract
import nltk

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from konlpy.tag import *
from collections import Counter
from nltk.corpus import stopwords
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [6]:
df = pd.read_csv('answers.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,id,author_id,question_id,content,tag_string,created_at,updated_at
0,1,5,236,"오늘 상담쌤과 칭찬에 대한 대화를 나눴다. 내가 애들의 칭찬을 잘 못 믿는, 그리고...",NaN,2019-01-28 14:11:42.543635,2019-01-28 14:11:42.543635
1,2,5,103,원하는 순간에 생각의 흐름을 멈추고 하고자 하는 일에 집중할 수 있는 능력.,NaN,2019-01-28 14:12:09.623446,2019-01-28 14:12:09.623446
2,3,5,98,가만히 공상/유튜브. 혼자 있는 시간.,NaN,2019-01-28 14:13:16.547800,2019-01-28 14:13:16.547800
3,4,5,95,게으름. 그 중에서도 요즘 가장 싫은건 게으름으로 인해 더러워진 방. 너무 짜증나서...,NaN,2019-01-28 14:14:38.838070,2019-01-28 14:14:38.838070
4,6,4,5,진수니랑 같이 프론트 더 멋있게 만들기!!!!!\r\n,NaN,2019-01-28 15:55:14.088330,2019-01-28 15:55:14.088330


In [7]:
twitter = Okt()

text = ""
morphs = []

content = df.content

for row in content:
    text = row
    if not pd.isna(text):
        morphs.append(twitter.pos(text))

In [4]:
tf_vect = TfidfVectorizer(tokenizer=getNVM_lemma,ngram_range=(1, 2), min_df=2, max_df=20000) 
dtm = tf_vect.fit_transform(df['content']) 

n_topics = 20

lda = LatentDirichletAllocation(n_components=n_topics) 
lda.fit(dtm)

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
names = tf_vect.get_feature_names() 
topics = dict() 

for idx, topic in enumerate(lda.components_): 
    vocab = [] 
    for i in topic.argsort()[:-(30-1):-1]: 
        vocab.append((names[i], topic[i].round(2))) 
    print("주제 %d:" % (idx +1)) 
    print([(names[i], topic[i].round(2)) for i in topic.argsort()[:-(30-1):-1]])

In [ ]:
visual = pyLDAvis.sklearn.prepare(lda_model=lda, dtm=dtm, vectorizer=tf_vect) 
pyLDAvis.save_html(visual, 'LDA_Visualization.html') 
pyLDAvis.display(visual)